# importing BERT model

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel


/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
save_path = "ruben_small_bert"

In [3]:
model = AutoModel.from_pretrained(save_path, return_dict=True)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(save_path)

In [5]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)
# (312,)

(312,)


# import and tokenize dataset

In [6]:
import pandas as pd

data = pd.read_csv('ru_toxic_dataset.csv')

In [7]:
data

,comment,toxic
0,дворника надо тоже уничтожить!,1.0
1,"моя старшая неделю шипела, не принимала подкид...",0.0
2,полностью с вами согласна!,0.0
3,"хоть ногу вверх, ничего не изменится",0.0
4,а что значит - левого ребенка?,0.0
...,...,...
163182,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
163183,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
163184,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
163185,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [8]:
data.toxic.sum()/data.toxic.size

0.1924601837156146

In [8]:
data["tokens"] = data.apply(lambda x: embed_bert_cls(x.comment, model, tokenizer), axis=1)

In [9]:
data.head()

,comment,toxic,tokens
0,дворника надо тоже уничтожить!,1.0,"[0.0059794295, -0.052715674, -0.05447943, -0.0..."
1,"моя старшая неделю шипела, не принимала подкид...",0.0,"[0.07568508, 0.04260101, 0.032866668, -0.03516..."
2,полностью с вами согласна!,0.0,"[-0.025587833, -0.036478575, -0.00693169, -0.0..."
3,"хоть ногу вверх, ничего не изменится",0.0,"[-0.0053351405, -0.011288563, 0.022511166, -0...."
4,а что значит - левого ребенка?,0.0,"[0.08475358, -0.005457756, -0.0123618785, -0.1..."


In [10]:
import numpy as np

In [11]:
features = np.stack(data.tokens.values)

In [12]:
labels = data.toxic.values

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2)

# Expeiments

In [14]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
lr_clf.score(test_features, test_labels)

0.8639928917213064

In [36]:
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]


In [44]:
from sklearn.model_selection import GridSearchCV
logModel = LogisticRegression(max_iter = 2500)
clf = GridSearchCV(logModel, param_grid = param_grid, cv = 5, verbose=True, n_jobs=4)

best_clf = clf.fit(train_features, train_labels)

Fitting 5 folds for each of 960 candidates, totalling 4800 fits
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TO

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documenta

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documenta

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linea

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/rub

...

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/ruben/my_project_dir/my_project_env/lib/pyt

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/rub

KeyboardInterrupt: 

In [45]:
best_clf.best_estimator_

LogisticRegression(C=3792.690190732246, max_iter=2500)

In [16]:
best_clf = LogisticRegression(C=3792.690190732246, max_iter=2500)

In [17]:
best_clf.fit(train_features, train_labels)
best_clf.score(test_features, test_labels)

0.868650039830872

In [26]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 2), random_state=1)
clf.fit(train_features, train_labels)

/home/ruben/my_project_dir/my_project_env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(4, 2), random_state=1,
              solver='lbfgs')

In [27]:
clf.score(test_features, test_labels)

0.8579263435259513

In [30]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,max_depth=6, random_state=0).fit(train_features, train_labels)

In [31]:
clf.score(test_features, test_labels)

0.8301672896623568

In [39]:
best_clf.predict(embed_bert_cls('', model, tokenizer).reshape(1, -1))

array([1.])

In [40]:
import pickle

filename = 'finalized_model.sav'
pickle.dump(best_clf, open(filename, 'wb'))

# Balance the classes!!!

In [37]:
data.toxic.sum()/data.toxic.count()

0.1924601837156146

In [13]:
d = data.groupby('toxic')

In [17]:
d.groups.values()
frames_of_groups = [x.sample(d.size().min()) for y, x in d]

In [19]:
new_data = pd.concat(frames_of_groups)

In [26]:
new_data.groupby('toxic').count()

,comment
toxic,
0.0,31407
1.0,31407


In [28]:
new_data["tokens"] = new_data.apply(lambda x: embed_bert_cls(x.comment, model, tokenizer), axis=1)

In [29]:
import numpy as np
features = np.stack(new_data.tokens.values)
labels = new_data.toxic.values

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2)

In [34]:
lr_clf = LogisticRegression(C=3792.690190732246,max_iter=5000)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=3792.690190732246, max_iter=5000)

In [35]:
lr_clf.score(test_features, test_labels)

0.8053012815410332